In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
credits = pd.read_csv('Data/tmdb_5000_credits.csv')
movies = pd.read_csv('Data/tmdb_5000_movies.csv')

In [3]:
credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [4]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [5]:
credits.columns = ['id', 'title', 'cast', 'crew']
moview = movies.merge(credits, on='id')

In [6]:
movies['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [7]:
# replace NaN values with empty strings
movies['overview'] = movies['overview'].fillna('')

In [8]:
# function -> using movie description and the keyword associated with the movie and genre to recommend the movie
def create_soup(x):
    return ''.join(x['keywords']) + '' + ''.join(x['genres']) + '' + ''.join(x['overview'])

# apply allow the users to pass a function and apply it on every single value of the Pandas series
movies['soup'] = movies.apply(create_soup, axis=1)

In [9]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['soup'])
tfidf_matrix.shape

(4803, 32768)

In [10]:
consine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [12]:
def get_recommendation(title, consine_sim):
    # get the index of the movie that match the title
    idx = indices[title]

    # get the pairwise similarity scores of all the movies
    sim_scores = list(enumerate(consine_sim[idx]))

    # sort the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1: 11]

    movie_indices = [i[0] for i in sim_scores]

    return movies['title'].iloc[movie_indices]

In [13]:
get_recommendation('The Avengers', consine_sim)

7                  Avengers: Age of Ultron
26              Captain America: Civil War
64                       X-Men: Apocalypse
242                         Fantastic Four
511                                  X-Men
79                              Iron Man 2
85     Captain America: The Winter Soldier
169     Captain America: The First Avenger
182                                Ant-Man
68                                Iron Man
Name: title, dtype: object